In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

path = "../../../dane/8CPU_20RAM/3600s/4_merged.csv"

In [16]:
# Załaduj dane
df = pd.read_csv(path)

In [17]:
for col in ['replicaId', 'endpointUrl_methods']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [18]:
# Dodajemy skalowanie dla wybranych kolumn
features_to_scale = ['queueSizeForward_methods', 'queueSizeBack_methods',
                     'cpuUsage_stock', 'memoryUsage_stock',
                     'applicationTime_trading', 'databaseTime_trading',
                     'numberOfSellOffers_trading', 'numberOfBuyOffers_trading']
scaler = StandardScaler()
for col in features_to_scale:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [19]:
# Wybór cech
features = [
            # 'timestamp',
            # 'apiTime_methods',
            # 'applicationTime_methods',
            # 'databaseTime_methods',
            'endpointUrl_methods',
            'queueSizeForward_methods',
            'queueSizeBack_methods',
            'cpuUsage_stock',
            'memoryUsage_stock',
            'applicationTime_trading',
            'databaseTime_trading',
            'numberOfSellOffers_trading',
            'numberOfBuyOffers_trading',
            # 'cpuUsage_traffic',
            # 'memoryUsage_traffic',
            # 'replicaId'
            ]
df_features = df[features]

# Kodowanie pudełek
le = LabelEncoder()
df['test'] = le.fit_transform(df['test'])

# Kopiowanie danych
df_encoded = df_features.copy()
df_encoded['test'] = df['test']

In [20]:
# Stwórz puste listy do przechowywania danych treningowych i testowych
X_train = []
X_test = []
y_train = []
y_test = []

In [21]:
def create_windows(X, y, window_size, step_size):
    X_windows = []
    y_windows = []

    # Przesuń okno po danych
    for i in range(0, len(X) - window_size, step_size):
        # Utwórz okno danych
        X_window = X.iloc[i:i + window_size]
        # Utwórz etykietę dla okna (etykieta ostatniej obserwacji w oknie)
        y_window = y.iloc[i + window_size]

        X_windows.append(X_window.values)
        y_windows.append(y_window)

    return np.array(X_windows), np.array(y_windows)

In [22]:
window_size = 700
step_size = 100
# Dla każdego unikalnego pudełka
for box in df_encoded['test'].unique():
    # Wybierz tylko rekordy dla tego pudełka
    box_data = df_encoded[df_encoded['test'] == box]
    
    # Oblicz punkt podziału (80% danych)
    split_point = int(len(box_data) * 0.8)
    
    # Dodaj pierwsze 80% rekordów do danych treningowych
    X_train_box = box_data.drop('test', axis=1).iloc[:split_point]
    y_train_box = box_data['test'].iloc[:split_point]
    
    # Dodaj ostatnie 20% rekordów do danych testowych
    X_test_box = box_data.drop('test', axis=1).iloc[split_point:]
    y_test_box = box_data['test'].iloc[split_point:]
    
    # Stwórz okienka dla danych treningowych
    X_train_windows, y_train_windows = create_windows(X_train_box, y_train_box, window_size, step_size)
    
    # Stwórz okienka dla danych testowych
    X_test_windows, y_test_windows = create_windows(X_test_box, y_test_box, window_size, step_size)
    
    # Dodaj okienka do list
    X_train.append(X_train_windows)
    y_train.append(y_train_windows)
    X_test.append(X_test_windows)
    y_test.append(y_test_windows)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from keras.utils import to_categorical

In [24]:
# Połączenie danych treningowych dla wszystkich pudełek
X_train_combined = np.concatenate(X_train, axis=0)
y_train_combined = np.concatenate(y_train, axis=0)

In [25]:
# Zakodowanie etykiet w formacie one-hot encoding
y_train_combined_encoded = to_categorical(y_train_combined, num_classes=len(df['test'].unique()))

In [26]:
# Trenowanie modelu na połączonym zestawie danych
model = Sequential()    
model.add(GRU(100, return_sequences=False, input_shape=(window_size, X_train_combined.shape[2])))
model.add(Dense(len(df['test'].unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined_encoded, epochs=5, batch_size=32)

Epoch 1/5
552/552 [==============================] - 178s 319ms/step - loss: 0.4055 - accuracy: 0.8557
Epoch 2/5
552/552 [==============================] - 178s 322ms/step - loss: 0.0562 - accuracy: 0.9839
Epoch 3/5
552/552 [==============================] - 213s 386ms/step - loss: 0.0310 - accuracy: 0.9916
Epoch 4/5
552/552 [==============================] - 217s 393ms/step - loss: 0.0075 - accuracy: 0.9984
Epoch 5/5
552/552 [==============================] - 216s 391ms/step - loss: 0.0132 - accuracy: 0.9958


In [27]:
for i in range(len(X_test)):
    y_test_encoded = to_categorical(y_test[i], num_classes=len(df['test'].unique()))
    loss, accuracy = model.evaluate(X_test[i], y_test_encoded)
    print(f"Test {i+1}: Loss = {loss}, Accuracy = {accuracy}")

31/31 [==============================] - 5s 117ms/step - loss: 0.0435 - accuracy: 0.9705
Test 1: Loss = 0.04346351698040962, Accuracy = 0.9704985022544861
29/29 [==============================] - 4s 126ms/step - loss: 0.0010 - accuracy: 1.0000
Test 2: Loss = 0.0010168211301788688, Accuracy = 1.0
39/39 [==============================] - 5s 118ms/step - loss: 4.9857e-04 - accuracy: 1.0000
Test 3: Loss = 0.0004985700943507254, Accuracy = 1.0
40/40 [==============================] - 5s 115ms/step - loss: 1.6902e-04 - accuracy: 1.0000
Test 4: Loss = 0.000169018967426382, Accuracy = 1.0
